In [6]:
%pwd

'/home/grkmkola/Desktop/Projects/mlops-proje/kidney-disease-classification/research'

In [7]:
%cd ..
%pwd

/home/grkmkola/Desktop/Projects/mlops-proje/kidney-disease-classification


/home/grkmkola/miniconda3/envs/kidney/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/home/grkmkola/Desktop/Projects/mlops-proje/kidney-disease-classification'

In [16]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    tensorboard_log_dir: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_augmentation: bool
    params_image_size: list
    params_early_stopping_patience: int
    params_learning_rate: float

In [17]:
import os
from cnnClassifier.constants import *
from cnnClassifier.utils import read_yaml, create_directories

In [19]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
        ) -> None:
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories(
            [
                self.config.artifacts_root,
                self.config.training.root_dir,
                self.config.training.tensorboard_log_dir,
            ]
        )

    def get_training_config(self):
        config = self.config.training
        prepare_base_model = self.config.prepare_base_model
        data_ingestion = self.config.data_ingestion

        params = self.params

        training_data = os.path.join(
            data_ingestion.unzip_dir,
            "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone"    
        )

        training_config = TrainingConfig(
            root_dir=Path(config.root_dir),
            trained_model_path=Path(config.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_early_stopping_patience=params.EARLY_STOPPING_PATIENCE,
            params_learning_rate=params.LEARNING_RATE
        )
        
        return training_config

In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from tqdm import tqdm
from torch.cuda.amp import GradScaler, autocast
from torch.utils.tensorboard import SummaryWriter
import logging
from pathlib import Path
from sklearn.metrics import precision_score, recall_score, f1_score

In [41]:
class Training:
    def __init__(self, config):
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.setup_logging()
        self.writer = SummaryWriter(log_dir=self.config.tensorboard_log_dir)
        self.scaler = GradScaler()

    def setup_logging(self):
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        self.logger = logging.getLogger()

    def get_base_model(self):
        self.model = torch.load(self.config.updated_base_model_path)
        self.model.to(self.device)

    def train_valid_loader(self):
        basic_transform = transforms.Compose([
            transforms.Resize(self.config.params_image_size[:-1]),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        if self.config.params_augmentation:
            train_transform = transforms.Compose([
                transforms.RandomRotation(40),
                transforms.RandomHorizontalFlip(),
                transforms.RandomResizedCrop(self.config.params_image_size[0], scale=(0.8, 1.0)),
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
                basic_transform
            ])
        else:
            train_transform = basic_transform

        full_dataset = datasets.ImageFolder(self.config.training_data, transform=train_transform)

        total_size = len(full_dataset)
        train_size = int(0.8 * total_size)
        valid_size = total_size - train_size

        train_dataset, valid_dataset = random_split(full_dataset, [train_size, valid_size], generator=torch.Generator().manual_seed(42))

        train_dataset.dataset.transform = train_transform
        valid_dataset.dataset.transform = basic_transform

        self.train_loader = DataLoader(train_dataset, batch_size=self.config.params_batch_size, shuffle=True, num_workers=4)
        self.valid_loader = DataLoader(valid_dataset, batch_size=self.config.params_batch_size, shuffle=False, num_workers=4)

        self.logger.info(f"Number of training samples: {len(train_dataset)}")
        self.logger.info(f"Number of validation samples: {len(valid_dataset)}")

    @staticmethod
    def save_model(path: Path, model: nn.Module):
        torch.save(model.state_dict(), path)

    def find_best_lr(self):
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters(), lr=1e-7)

        lrs = []
        losses = []
        best_loss = float('inf')
        smoothing = 0.05

        for inputs, labels in tqdm(self.train_loader, desc="Finding best LR"):
            inputs, labels = inputs.to(self.device), labels.to(self.device)

            optimizer.zero_grad()
            with autocast():
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
            self.scaler.scale(loss).backward()
            self.scaler.step(optimizer)
            self.scaler.update()

            # Track the learning rate and loss
            lrs.append(optimizer.param_groups[0]["lr"])
            losses.append(loss.item())
            optimizer.param_groups[0]["lr"] *= 1.1  # Increase the learning rate

            # Smooth the loss for stability
            if loss.item() < best_loss:
                best_loss = loss.item()
            else:
                loss = smoothing * loss.item() + (1 - smoothing) * best_loss

        self.writer.add_scalar('LR Finder/lr', lrs, 0)
        self.writer.add_scalar('LR Finder/loss', losses, 0)
        best_lr = lrs[losses.index(min(losses))]
        return best_lr

    def train(self):
        best_lr = self.find_best_lr()
        self.logger.info(f"Using best LR: {best_lr}")
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters(), lr=best_lr)
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=best_lr, steps_per_epoch=len(self.train_loader), epochs=self.config.params_epochs)
        best_valid_loss = float('inf')
        early_stopping_counter = 0

        for epoch in range(self.config.params_epochs):
            self.model.train()
            train_loss = 0.0
            train_pbar = tqdm(self.train_loader, desc=f'Epoch {epoch+1}/{self.config.params_epochs} [Train]')
            for inputs, labels in train_pbar:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                optimizer.zero_grad()
                with autocast():
                    outputs = self.model(inputs)
                    loss = criterion(outputs, labels)
                self.scaler.scale(loss).backward()
                self.scaler.step(optimizer)
                self.scaler.update()
                scheduler.step()

                train_loss += loss.item() * inputs.size(0)
                train_pbar.set_postfix({'loss': f'{loss.item():.4f}'})

            train_loss = train_loss / len(self.train_loader.dataset)
            self.writer.add_scalar('Loss/train', train_loss, epoch)

            self.model.eval()
            valid_loss = 0.0
            correct = 0
            total = 0
            all_labels = []
            all_preds = []
            valid_pbar = tqdm(self.valid_loader, desc=f'Epoch {epoch+1}/{self.config.params_epochs} [Valid]')
            with torch.no_grad():
                for inputs, labels in valid_pbar:
                    inputs, labels = inputs.to(self.device), labels.to(self.device)
                    outputs = self.model(inputs)
                    loss = criterion(outputs, labels)
                    valid_loss += loss.item() * inputs.size(0)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                    valid_pbar.set_postfix({'loss': f'{loss.item():.4f}'})

                    all_labels.extend(labels.cpu().numpy())
                    all_preds.extend(predicted.cpu().numpy())

            valid_loss = valid_loss / len(self.valid_loader.dataset)
            accuracy = 100 * correct / total
            precision = precision_score(all_labels, all_preds, average='weighted')
            recall = recall_score(all_labels, all_preds, average='weighted')
            f1 = f1_score(all_labels, all_preds, average='weighted')

            self.writer.add_scalar('Loss/valid', valid_loss, epoch)
            self.writer.add_scalar('Accuracy/valid', accuracy, epoch)
            self.writer.add_scalar('Precision/valid', precision, epoch)
            self.writer.add_scalar('Recall/valid', recall, epoch)
            self.writer.add_scalar('F1-Score/valid', f1, epoch)

            self.logger.info(f'Epoch {epoch+1}/{self.config.params_epochs}, '
                             f'Train Loss: {train_loss:.4f}, '
                             f'Valid Loss: {valid_loss:.4f}, '
                             f'Valid Accuracy: {accuracy:.2f}%, '
                             f'Precision: {precision:.4f}, '
                             f'Recall: {recall:.4f}, '
                             f'F1-Score: {f1:.4f}')

            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                early_stopping_counter = 0
                self.save_model(path=self.config.trained_model_path, model=self.model)
                self.logger.info(f'Saved model with valid loss: {valid_loss:.4f}')
            else:
                early_stopping_counter += 1
                if early_stopping_counter >= self.config.early_stopping_patience:
                    self.logger.info('Early stopping triggered')
                    break

        self.writer.close()


In [42]:
config = ConfigurationManager().get_training_config()
training = Training(config)
training.get_base_model()
training.train_valid_loader()
training.train()

[2024-07-23 13:46:16,969: INFO: utils: yaml file config/config.yaml loaded successfully:]
[2024-07-23 13:46:16,971: INFO: utils: yaml file params.yaml loaded successfully:]
[2024-07-23 13:46:16,972: INFO: utils: created directory at: artifacts:]
[2024-07-23 13:46:16,973: INFO: utils: created directory at: artifacts/training:]


AttributeError: 'TrainingConfig' object has no attribute 'tensorboard_log_dir'